In [1]:
from shared_code.utility.spark.set_environ import *
import os
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

set_azure_env()

from pyspark.sql.types import StringType, BooleanType
from pyspark.sql.functions import udf
from shared_code.utility.storage.table import TableAdapter
from shared_code.utility.schemas.spark_table_schema import *


spark = get_session("normalize-paths")

In [4]:
@udf(returnType=StringType())
def get_missing_name(path):
	try:
		return os.path.split(path)[-1]
	except:
		return ""

@udf(returnType=StringType())
def get_correct_path(path, sub, name):
	normal_path = os.path.normpath(path)
	if "reddit_images" in normal_path:
		updated_path = os.path.join("D:", os.sep, "data", "images", sub, name)
		return updated_path
	if "raw_reddit_images" in normal_path:
		updated_path = os.path.join("D:", os.sep, "data", "images", sub, name)
		return updated_path
	else:
		updated_path = os.path.join("D:", os.sep, "data", "images", sub, name)
		return updated_path

@udf(returnType=BooleanType())
def check_if_exists(path):
	return os.path.exists(path)

@udf(returnType=BooleanType())
def check_if_curated(curated: bool):
	if curated is None:
		return False
	if curated == "True":
		return True
	else:
		return False

@udf(returnType=StringType())
def get_thumbnail_path(path:str, sub:str, name:str):
	print(f"Checking thumbnail path: {path} for {sub} and {name}")
	if path == "" or path is None or path == "None" or path == "NaN":
		expected_path = os.path.join("D:", os.sep, "data", "images", sub, "thumbnails", name)
		if os.path.exists(expected_path):
			print(f"Found thumbnail as expected: {expected_path}")
			return expected_path
		else:
			return ""
	else:
		if os.path.exists(path):
			print(f"Found thumbnail already: {path}")
			return path
		else:
			return ""

In [10]:
table_name = "training"
table_broker: TableAdapter = TableAdapter()

raw_data = table_broker.get_all_entities(table_name)
df = spark.createDataFrame(raw_data, schema=image_table_schema)


named_frame = df.withColumn("image_name", get_missing_name("image"))

print(f"Total records From Cloud: {named_frame.count()}")
display(named_frame.limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated


In [12]:
if named_frame.count() == 0:
	print("No records to update")
else:
	records = named_frame.toPandas().to_dict(orient='records')
	print(f"Total records to update: {named_frame.count()}")
	for i, record in enumerate(records):
		if i % 100 == 0:
			print(f"Updating record {i} of {named_frame.count()}")
		table_broker.upsert_entity_to_table(table_name, record)

Total records to update: 0
No records to update


In [13]:
entities = list(table_broker.get_all_entities(table_name))
data_frame = spark.createDataFrame(entities, schema=image_table_schema)
print(f"Total records From Clound: {data_frame.count()}")
display(data_frame.limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,New York in the fog,1000cej,OtterlyFoxy,https://i.redd.it/4emw5uldib9a1.jpg,NaN,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,7a8d96e378c15c8ab8440ac311f12c11,CityPorn,A person is riding a bicycle on a city street.,True,4emw5uldib9a1.jpg,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,False
1,training,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,Thoughts about my NYE party outfit?,1000d16,princessxo699,https://i.imgur.com/GgFEagO.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,9951b4f82caeb8ba2bd9f79f8d422450,SFWNextDoorGirls,A little girl in a pink dress is holding a pin...,True,GgFEagO.jpg,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,False
2,training,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,(IKTR),1000fg0,BlkBrd1312,https://i.redd.it/nwa7hts2jb9a1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/1000fg0/iktr/,be8dd55e34216bec1e15e03fa296eacc,HotGirlNextDoor,A woman in a pink bathing suit holds a baby in...,True,nwa7hts2jb9a1.jpg,a woman in a bathing suit with a cell phone in...,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,False
3,training,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,Just looking for entertainment,1000glf,toolate_sharkbait,https://i.redd.it/4xyb1vgbjb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1000glf/just_looking_for_...,e554c1ed7ffa2740436ac082068b2824,AmIhotAF,A woman in a pink dress is smiling at the camera.,True,4xyb1vgbjb9a1.jpg,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,False
4,training,1000j1n,/data/images/greentext/3mewbe0wjb9a1.jpg,Anon wants Elon cut,1000j1n,trent8051,https://i.redd.it/3mewbe0wjb9a1.jpg,None,/r/greentext/comments/1000j1n/anon_wants_elon_...,1dec3dabb5e46cde01855d06089c287a,greentext,A man and a woman are sitting at a table with ...,True,3mewbe0wjb9a1.jpg,a collage of photos showing a man in a suit an...,/data/images/greentext/thumbnail/3mewbe0wjb9a1...,False
5,training,1000qiq,D:\data\images\SFWRedheads\0mgrt95plb9a1.jpg,Who’s going to be my new years kiss tonight? 🤧,1000qiq,fireybubbulegum,https://i.redd.it/0mgrt95plb9a1.jpg,NaN,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,7b0be38a08e3316051c6e5a3753bc791,SFWRedheads,A woman in a red dress is talking to a man in ...,False,0mgrt95plb9a1.jpg,NaN,None,False
6,training,1001fon,D:\data\images\SFWRedheads\gp4fhsdorb9a1.jpg,honk if you like my sweater 😁💕,1001fon,glam_pire,https://i.redd.it/gp4fhsdorb9a1.jpg,NaN,/r/SFWRedheads/comments/1001fon/honk_if_you_li...,feed678c3b74ed1d0b7011f28e8f283f,SFWRedheads,A woman in a red dress is talking to a man in ...,False,gp4fhsdorb9a1.jpg,NaN,None,False
7,training,1001ghp,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,Braids,1001ghp,eromorphic,https://i.redd.it/v3ra9g4vrb9a1.jpg,NaN,/r/sfwpetite/comments/1001ghp/braids/,16ca1536c4f6936d3e994df7cb40ab8e,sfwpetite,A girl in a pink bathing suit walks down the b...,True,v3ra9g4vrb9a1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,False
8,training,1001ih5,D:\data\images\SFWNextDoorGirls\bhigr5rbsb9a1.jpg,New Year's Elf,1001ih5,ms_twine_grace,https://i.redd.it/bhigr5rbsb9a1.jpg,NaN,/r/SFWNextDoorGirls/comments/1001ih5/new_years...,9ff18e9afb7b566c0d50add45cbfb989,SFWNextDoorGirls,A little girl in a pink dress is dancing on a ...,False,bhigr5rbsb9a1.jpg,NaN,None,False
9,training,1001kp4,D:\data\images\AmIhotAF\dn7lcclvsb9a1.jpg,Happy New Years Eve,1001kp4,Hotblondie69420,https://i.redd.it/dn7lcclvsb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1001kp4/happy_new_years_eve/,7137ef4fe19a97c757553bfa62b23a33,AmIhotAF,A woman in a white dress is talking to a man i...,False,dn7lcclvsb9a1.jpg,NaN,N

In [36]:
max_attempts = 5

out_data = None

while max_attempts > 0:
	try:
		out_data = data_frame\
			.withColumn("vimage", get_correct_path("image", "subreddit", "image_name"))\
			.withColumn("vexists", check_if_exists("image"))\
			.withColumn("vcurated", check_if_curated("curated"))\
			.withColumn("vsmall_image", get_thumbnail_path("small_image", "subreddit", "image_name"))\
			.drop("image")\
			.drop("exists")\
			.drop("curated")\
			.drop("small_image")\
			.withColumnRenamed("vimage", "image")\
			.withColumnRenamed("vexists", "exists")\
			.withColumnRenamed("vcurated", "curated")\
			.withColumnRenamed("vsmall_image", "small_image")\
			.select("PartitionKey", "RowKey","image", "text", "id", "author", "url", "flair", "permalink", "hash", "subreddit", "caption", "exists", "image_name", "updated_caption", "small_image", "curated") \
			.collect()
		break
	except:
		max_attempts -= 1
		print(f"Failed to update paths, attempts remaining: {max_attempts}")



In [37]:
final_df = spark.createDataFrame(out_data, schema=image_table_schema)

print("=== Normalized Path ===")
display(final_df.limit(10).toPandas())

records = final_df.toPandas().to_dict(orient='records')
print(f"Total records to update: {final_df.count()}")
for record in records:
	table_broker.upsert_entity_to_table(table_name, record)

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,New York in the fog,1000cej,OtterlyFoxy,https://i.redd.it/4emw5uldib9a1.jpg,NaN,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,7a8d96e378c15c8ab8440ac311f12c11,CityPorn,A person is riding a bicycle on a city street.,True,4emw5uldib9a1.jpg,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,False
1,training,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,Thoughts about my NYE party outfit?,1000d16,princessxo699,https://i.imgur.com/GgFEagO.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,9951b4f82caeb8ba2bd9f79f8d422450,SFWNextDoorGirls,A little girl in a pink dress is holding a pin...,True,GgFEagO.jpg,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,False
2,training,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,(IKTR),1000fg0,BlkBrd1312,https://i.redd.it/nwa7hts2jb9a1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/1000fg0/iktr/,be8dd55e34216bec1e15e03fa296eacc,HotGirlNextDoor,A woman in a pink bathing suit holds a baby in...,True,nwa7hts2jb9a1.jpg,a woman in a bathing suit with a cell phone in...,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,False
3,training,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,Just looking for entertainment,1000glf,toolate_sharkbait,https://i.redd.it/4xyb1vgbjb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1000glf/just_looking_for_...,e554c1ed7ffa2740436ac082068b2824,AmIhotAF,A woman in a pink dress is smiling at the camera.,True,4xyb1vgbjb9a1.jpg,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,False
4,training,1000j1n,D:\data\images\greentext\3mewbe0wjb9a1.jpg,Anon wants Elon cut,1000j1n,trent8051,https://i.redd.it/3mewbe0wjb9a1.jpg,None,/r/greentext/comments/1000j1n/anon_wants_elon_...,1dec3dabb5e46cde01855d06089c287a,greentext,A man and a woman are sitting at a table with ...,True,3mewbe0wjb9a1.jpg,a collage of photos showing a man in a suit an...,/data/images/greentext/thumbnail/3mewbe0wjb9a1...,False
5,training,1000qiq,D:\data\images\SFWRedheads\0mgrt95plb9a1.jpg,Who’s going to be my new years kiss tonight? 🤧,1000qiq,fireybubbulegum,https://i.redd.it/0mgrt95plb9a1.jpg,NaN,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,7b0be38a08e3316051c6e5a3753bc791,SFWRedheads,A woman in a red dress is talking to a man in ...,False,0mgrt95plb9a1.jpg,NaN,,False
6,training,1001fon,D:\data\images\SFWRedheads\gp4fhsdorb9a1.jpg,honk if you like my sweater 😁💕,1001fon,glam_pire,https://i.redd.it/gp4fhsdorb9a1.jpg,NaN,/r/SFWRedheads/comments/1001fon/honk_if_you_li...,feed678c3b74ed1d0b7011f28e8f283f,SFWRedheads,A woman in a red dress is talking to a man in ...,False,gp4fhsdorb9a1.jpg,NaN,,False
7,training,1001ghp,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,Braids,1001ghp,eromorphic,https://i.redd.it/v3ra9g4vrb9a1.jpg,NaN,/r/sfwpetite/comments/1001ghp/braids/,16ca1536c4f6936d3e994df7cb40ab8e,sfwpetite,A girl in a pink bathing suit walks down the b...,True,v3ra9g4vrb9a1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,False
8,training,1001ih5,D:\data\images\SFWNextDoorGirls\bhigr5rbsb9a1.jpg,New Year's Elf,1001ih5,ms_twine_grace,https://i.redd.it/bhigr5rbsb9a1.jpg,NaN,/r/SFWNextDoorGirls/comments/1001ih5/new_years...,9ff18e9afb7b566c0d50add45cbfb989,SFWNextDoorGirls,A little girl in a pink dress is dancing on a ...,False,bhigr5rbsb9a1.jpg,NaN,,False
9,training,1001kp4,D:\data\images\AmIhotAF\dn7lcclvsb9a1.jpg,Happy New Years Eve,1001kp4,Hotblondie69420,https://i.redd.it/dn7lcclvsb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1001kp4/happy_new_years_eve/,7137ef4fe19a97c757553bfa62b23a33,AmIhotAF,A woman in a white dress is talking to a man i...,False,dn7lcclvsb9a1.jpg,NaN,,False


In [39]:
writable_df = final_df
print(f"Total records to write: {writable_df.count()}")
display(writable_df.limit(10).toPandas())
writable_df.write.parquet("D:\\data\\processed\\reddit_images.parquet", mode="overwrite")

Total records to write: 7190


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,New York in the fog,1000cej,OtterlyFoxy,https://i.redd.it/4emw5uldib9a1.jpg,NaN,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,7a8d96e378c15c8ab8440ac311f12c11,CityPorn,A person is riding a bicycle on a city street.,True,4emw5uldib9a1.jpg,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,False
1,training,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,Thoughts about my NYE party outfit?,1000d16,princessxo699,https://i.imgur.com/GgFEagO.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,9951b4f82caeb8ba2bd9f79f8d422450,SFWNextDoorGirls,A little girl in a pink dress is holding a pin...,True,GgFEagO.jpg,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,False
2,training,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,(IKTR),1000fg0,BlkBrd1312,https://i.redd.it/nwa7hts2jb9a1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/1000fg0/iktr/,be8dd55e34216bec1e15e03fa296eacc,HotGirlNextDoor,A woman in a pink bathing suit holds a baby in...,True,nwa7hts2jb9a1.jpg,a woman in a bathing suit with a cell phone in...,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,False
3,training,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,Just looking for entertainment,1000glf,toolate_sharkbait,https://i.redd.it/4xyb1vgbjb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1000glf/just_looking_for_...,e554c1ed7ffa2740436ac082068b2824,AmIhotAF,A woman in a pink dress is smiling at the camera.,True,4xyb1vgbjb9a1.jpg,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,False
4,training,1000j1n,D:\data\images\greentext\3mewbe0wjb9a1.jpg,Anon wants Elon cut,1000j1n,trent8051,https://i.redd.it/3mewbe0wjb9a1.jpg,None,/r/greentext/comments/1000j1n/anon_wants_elon_...,1dec3dabb5e46cde01855d06089c287a,greentext,A man and a woman are sitting at a table with ...,True,3mewbe0wjb9a1.jpg,a collage of photos showing a man in a suit an...,/data/images/greentext/thumbnail/3mewbe0wjb9a1...,False
5,training,1000qiq,D:\data\images\SFWRedheads\0mgrt95plb9a1.jpg,Who’s going to be my new years kiss tonight? 🤧,1000qiq,fireybubbulegum,https://i.redd.it/0mgrt95plb9a1.jpg,NaN,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,7b0be38a08e3316051c6e5a3753bc791,SFWRedheads,A woman in a red dress is talking to a man in ...,False,0mgrt95plb9a1.jpg,NaN,,False
6,training,1001fon,D:\data\images\SFWRedheads\gp4fhsdorb9a1.jpg,honk if you like my sweater 😁💕,1001fon,glam_pire,https://i.redd.it/gp4fhsdorb9a1.jpg,NaN,/r/SFWRedheads/comments/1001fon/honk_if_you_li...,feed678c3b74ed1d0b7011f28e8f283f,SFWRedheads,A woman in a red dress is talking to a man in ...,False,gp4fhsdorb9a1.jpg,NaN,,False
7,training,1001ghp,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,Braids,1001ghp,eromorphic,https://i.redd.it/v3ra9g4vrb9a1.jpg,NaN,/r/sfwpetite/comments/1001ghp/braids/,16ca1536c4f6936d3e994df7cb40ab8e,sfwpetite,A girl in a pink bathing suit walks down the b...,True,v3ra9g4vrb9a1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,False
8,training,1001ih5,D:\data\images\SFWNextDoorGirls\bhigr5rbsb9a1.jpg,New Year's Elf,1001ih5,ms_twine_grace,https://i.redd.it/bhigr5rbsb9a1.jpg,NaN,/r/SFWNextDoorGirls/comments/1001ih5/new_years...,9ff18e9afb7b566c0d50add45cbfb989,SFWNextDoorGirls,A little girl in a pink dress is dancing on a ...,False,bhigr5rbsb9a1.jpg,NaN,,False
9,training,1001kp4,D:\data\images\AmIhotAF\dn7lcclvsb9a1.jpg,Happy New Years Eve,1001kp4,Hotblondie69420,https://i.redd.it/dn7lcclvsb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1001kp4/happy_new_years_eve/,7137ef4fe19a97c757553bfa62b23a33,AmIhotAF,A woman in a white dress is talking to a man i...,False,dn7lcclvsb9a1.jpg,NaN,,False


In [40]:
processed = spark.read.parquet("D:\\data\\processed\\reddit_images.parquet")
display(processed.limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11094it,D:\data\images\greentext\q0rxrphnuqha1.jpg,Anon is insulinpilled,11094it,TheGourmetGaylord,https://i.redd.it/q0rxrphnuqha1.jpg,None,/r/greentext/comments/11094it/anon_is_insulinp...,d39377ac12d266df304429d5b03ff483,greentext,None,True,q0rxrphnuqha1.jpg,None,/data/images/greentext/thumbnail/q0rxrphnuqha1...,False
1,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,a double decker bus is driving down the street,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
2,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,a woman in a bikini standing next to a tree,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
3,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,a cityscape of a city with tall buildings,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
4,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,a woman with glasses and a cat in her lap,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
5,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,a large building with a clock on top of it,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
6,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,a woman holding a cell phone in front of her face,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
7,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,a woman taking a picture of herself in a bathr...,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
8,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,a woman holding a bottle of beer next to a wall,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False
9,training,110c9o8,D:\data\images\CityPorn\ndc1hr2wvrha1.jpg,"Quetta, the Tucson of Pakistan",110c9

In [ ]:
print("Notebook Complete")